In [ ]:
import pandas as pd
import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
df = pd.read_csv('./input/spam.csv',delimiter=',',encoding='latin-1')
df.head()

In [ ]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

In [ ]:
sns.countplot(df.v1)
plt.xlabel('Label')
plt.title('Number of ham and spam messages')

In [ ]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
type(X_test)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)
print(test_sequences_matrix)

In [ ]:
model.predict(test_sequences_matrix)

In [ ]:
type(X_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
def modelPredict(predict, model):
    test_sequences = tok.texts_to_sequences(predict)
    test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
    return model.predict(test_sequences_matrix)

In [ ]:
testString = "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C\'s apply 08452810075over18\'s"
testString1= "Last Chance! Claim ur å£150 worth of discount vouchers today! Text SHOP to 85023 now! SavaMob, offers mobile! T Cs SavaMob POBOX84, M263UZ. å£3.00 Sub. 16"
testString2 = "This is a Scam Free entry Please mobile sms text call"
testString3 = "ICA Mobile Scam Call Text SMS"
testString4 = "dais hsa ijdhais as hdkjsah ka sahdhask"
seriess = pd.Series([testString1])
modelPredict(seriess, model)

In [ ]:
str(round(modelPredict(seriess, model).item(0)*100,2))

In [ ]:
model.save('model')

In [ ]:
X_test.to_csv('./X_test.csv', header=False)
X_train.to_csv('./X_train.csv', header=False)

In [ ]:
print(X_test)
type(X_test)

In [1]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string
df = pd.read_csv('./input/emails.csv')
df.drop_duplicates(inplace = True)
nltk.download('stopwords')
def process_text(text):
    '''
    What will be covered:
    1. Remove punctuation
    2. Remove stopwords
    3. Return list of clean text words
    '''
    
    #1
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    #2
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    #3
    return clean_words



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\multo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df['text'].head().apply(process_text)
from sklearn.feature_extraction.text import CountVectorizer
messages_bow = CountVectorizer(analyzer=process_text).fit_transform(df['text'])

In [3]:
print(df['text'])

0       Subject: naturally irresistible your corporate...
1       Subject: the stock trading gunslinger  fanny i...
2       Subject: unbelievable new homes made easy  im ...
3       Subject: 4 color printing special  request add...
4       Subject: do not have money , get software cds ...
                              ...                        
5723    Subject: re : research and development charges...
5724    Subject: re : receipts from visit  jim ,  than...
5725    Subject: re : enron case study update  wow ! a...
5726    Subject: re : interest  david ,  please , call...
5727    Subject: news : aurora 5 . 2 update  aurora ve...
Name: text, Length: 5695, dtype: object


In [4]:
from sklearn.model_selection import train_test_split
print(messages_bow)
X_train, X_test, y_train, y_test = train_test_split(messages_bow, df['spam'], test_size = 0.20, random_state = 0)
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

  (0, 3638)	1
  (0, 23369)	1
  (0, 18841)	1
  (0, 10065)	1
  (0, 17696)	1
  (0, 21140)	1
  (0, 27986)	1
  (0, 16674)	1
  (0, 28110)	1
  (0, 9296)	3
  (0, 21654)	2
  (0, 15429)	1
  (0, 32602)	1
  (0, 18238)	1
  (0, 18886)	1
  (0, 16089)	2
  (0, 8054)	1
  (0, 20952)	3
  (0, 32319)	1
  (0, 31968)	1
  (0, 24838)	1
  (0, 36025)	2
  (0, 21431)	2
  (0, 33037)	1
  (0, 23040)	2
  :	:
  (5694, 24818)	2
  (5694, 21624)	1
  (5694, 5729)	9
  (5694, 30934)	1
  (5694, 2828)	3
  (5694, 13338)	1
  (5694, 13127)	1
  (5694, 17388)	1
  (5694, 14130)	1
  (5694, 20273)	1
  (5694, 31827)	1
  (5694, 13128)	1
  (5694, 20467)	1
  (5694, 35288)	1
  (5694, 8629)	1
  (5694, 30082)	1
  (5694, 13522)	5
  (5694, 36185)	1
  (5694, 959)	2
  (5694, 2797)	1
  (5694, 30287)	1
  (5694, 17590)	1
  (5694, 33923)	1
  (5694, 10373)	1
  (5694, 11386)	1


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [5]:
#Evaluate the model on the training data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train ,pred ))
print('Confusion Matrix: \n',confusion_matrix(y_train,pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))
#Evaluate the model on the test data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(y_test ,pred ))
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

    accuracy                           1.00      4556
   macro avg       0.99      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556

Confusion Matrix: 
 [[3445   12]
 [   1 1098]]

Accuracy:  0.9971466198419666
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139

Confusion Matrix: 
 [[862   8]
 [  1 268]]

Accuracy:  0.9920983318700615


In [6]:
print(y_test)

977     1
3275    0
4163    0
751     1
3244    0
       ..
4506    0
1050    1
3366    0
2191    0
3911    0
Name: spam, Length: 1139, dtype: int64


In [7]:
message_boxx =  "tx sadasxzc zx c  xzc zc xt"
testString = process_text(message_boxx)
something = CountVectorizer(analyzer=process_text).fit_transform(testString['text'])
print(something)
classifier.predict(something)

TypeError: list indices must be integers or slices, not str

In [ ]:
xxxx= "tsaddsaxzcxxzcxz asdas   dsa  xzcxzcxz  xest"
xxxxx = process_text(xxxx)
stringggg = CountVectorizer(analyzer=process_text).fit_transform(pd.Series((i[0] for i in xxxxx)))
print(stringggg)